Ej. 3. Expansión de consultas

Primero instalamos todo el paquete BM25s ejecutando la siguiente orden:

In [ ]:
!pip install bm25s[full]

Ahora importamos las librerías Python necesarias para trabajar inicialmente

In [ ]:
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus

Descargamos la colección TREC-COVID

In [ ]:
!gdown 1Q1ZnHkstIIQQF8NTPyKQH7sEcpVfThFO
!unzip -o trec-covid-RI.zip

Seguimos con la preparación del corpus

In [ ]:
# Parseamos el corpus
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_lines = f.readlines() # Leemos todas las lineas del JSONL
    corpus_content = [] # Preparamos una lista de objetos JSON
    for linea in corpus_lines: # Cada línea de un JSONL es un objeto JSON...
        corpus_content.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Preparamos el corpus en los dos formatos requeridos
corpus_verbatim = list() # Una versión literal que mantenga la estructura original de los "documentos".
corpus_plaintext = list() # Una versión de texto plano para tokenización e indexación.
for entry in corpus_content:
    # BM25S expects 'id', 'title', and 'text'
    document = {"id": entry["_id"], "title": entry["title"].lower(), "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

Creamos un stemmer y tokenizamos el corpus

In [ ]:
# Tokenizamos mediante un stemmer para el idioma inglés (PREPROCESADO Y NORMALIZACIÓN)
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en", stemmer=stemmer, show_progress=True)

Creamos el retriever con la mejor parametrización obtenida del ejercicio anterior (Eliminación de palabras vacías, aplicando estematización y usando la variante de BM25s e IDF lucene) e indexamos el corpus ya tokenizado

In [ ]:
bm25_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF
idf_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF

retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True) # Indexamos el corpus tokenizado

Cargamos las consultas de la colección

In [ ]:
queries = []
with open("queries.jsonl", "r", encoding="utf-8") as f:
    queries_lines = f.readlines() # Leemos todas las lineas del JSONL
    for linea in queries_lines: # Cada línea de un JSONL es un objeto JSON...
        queries.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

Ahora implementamos el código para obtener el contenido textual de los n documentos relevantes y sus puntuaciones para cada consulta del conjunto de consultas cargado antes

In [ ]:
def submit_queries_and_get_run(queries, stemmer, retriever, stopwordsConfig, max_results=100, retornarPuntuaciones=False):
    run = {}

    # Iteramos para enviar cada consulta al retriever para obtener
    # resultados
    for query in queries:
        # Para evaluar después el rendimiento, es útil conservar el id de
        # la consulta
        query_id = query["_id"]

        query_string = query["metadata"]["narrative"].lower()

        # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
        # con la misma configuración que usamos al indexar el corpus.
        #
        query_tokenized = bm25s.tokenize(query_string, stopwords=stopwordsConfig, stemmer=stemmer, show_progress=False)

        # Retornamos los k resultados más “top” como una tupla con nombre.
        # Por favor, lee la documentación de BM25S para otras alternativas.
        #
        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        # Los documentos y sus puntuaciones se guardan en dos campos
        # diferentes de la tupla.
        #
        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_content = []
        for i in range(len(returned_documents)):
##          print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            if retornarPuntuaciones:
              returned_content.append([returned_documents[i], relevance_scores[i]])
            else:
              returned_content.append(returned_documents[i])
        run[query_id] = returned_content

    return run

Realizamos la llamada a la función anterior y almacenamos los resultados de la función llamada

In [ ]:
n = 10 # parámetro configurable
relevantes = submit_queries_and_get_run(queries, stemmer, retriever, "en", n)

contenidoTextual = [] # para cada query guardamos el contenido textual de los documentos resultado

for i in relevantes: # para cada query tenemos documentos...
    texto = ""
    for j in range(len(relevantes[i])):
      texto += relevantes[i][j]["text"]
    contenidoTextual.append(texto)

Desarrollamos una función que nos devuelva las frecuencias de aparición de los términos (TF)

In [ ]:
from collections import Counter

def calcularTF(texto, tokenizar):
    if tokenizar:
      tokens = bm25s.tokenize(texto, stopwords="en", stemmer=stemmer, show_progress=False)
    else:
      tokens = texto

    tmp = dict()

    for document in tokens[0]:
        freqs = dict(Counter(document))
        for token, freq in freqs.items():
            try:
                tmp[token] += freq
            except:
                tmp[token] = freq

    inverted_vocab = {tokens[1][key]: key for key in tokens[1].keys()}

    total_freqs = dict()

    for key, freq in dict(tmp).items():
        term = inverted_vocab[key]
        total_freqs[term] = freq

    return total_freqs

Hacemos la llamada a la función anterior y obtenemos las frecuencias de aparición para cada término

In [ ]:
frecuencias_relevantes = []
#indexParaImprimir = 0
for i in contenidoTextual:
  frecuencias_relevantes.append(calcularTF(i, True))
  #print(frecuencias_relevantes[indexParaImprimir])
  #indexParaImprimir += 1

Calculamos las frecuencias de aparición para cada término en todos los documentos del corpus

In [ ]:
# Calculamos las frecuencias de los términos en todos los documentos del corpus
frecuencias_corpus = calcularTF(corpus_tokenized, False)
#print(frecuencias_corpus)

A partir de las frecuencias de los términos de los documentos relevantes y las de los términos de todo el corpus, obtenemos las frecuencias de los términos del corpus que no aparecen en la lista de frecuencias de los documentos relevantes

In [ ]:
frecuencias_relevantes_restantes = []

for i in range(len(frecuencias_relevantes)):
  frecuencias_relevantes_restantes.append(dict())

# recorrer los terminos del corpus
for termino in frecuencias_corpus:
  frecuencia = frecuencias_corpus[termino]
  # para cada termino buscamos si esta en las frecuencias relevantes de cada query
  for i in range(len(frecuencias_relevantes)):
    conjunto_relevantes = frecuencias_relevantes[i]
    # si el termino del corpus no esta en dicho conjunto i-esimo
    if termino not in conjunto_relevantes.keys():
      # añadimos el termino junto con su frecuencia a la lista de frecuencias restantes en la posicion i-esima
      frecuencias_relevantes_restantes[i][termino] = frecuencia

#for i in range(len(frecuencias_relevantes_restantes)):
  #print(frecuencias_relevantes_restantes[i])

Desarrollamos el código que compute el método signed root log likelihood ratio test

In [ ]:
import math
from collections import Counter
from heapq import heappush, heappop
from typing import List, Tuple, Any


class LogLikelihood:
    @staticmethod
    def entropy(*elements: int) -> float:
        """
        Calculates the unnormalized Shannon entropy:
        -sum x_i log x_i / N = -N sum x_i/N log x_i/N
        """
        total = sum(elements)
        result = sum(LogLikelihood._x_log_x(e) for e in elements)
        return LogLikelihood._x_log_x(total) - result

    @staticmethod
    def _x_log_x(x: int) -> float:
        return 0.0 if x == 0 else x * math.log(x)

    @staticmethod
    def log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the raw log-likelihood ratio for two events.
        """
        assert k11 >= 0 and k12 >= 0 and k21 >= 0 and k22 >= 0, "Counts must be non-negative"

        row_entropy = LogLikelihood.entropy(k11 + k12, k21 + k22)
        column_entropy = LogLikelihood.entropy(k11 + k21, k12 + k22)
        matrix_entropy = LogLikelihood.entropy(k11, k12, k21, k22)

        if row_entropy + column_entropy < matrix_entropy:
            return 0.0  # Round-off error

        return 2.0 * (row_entropy + column_entropy - matrix_entropy)

    @staticmethod
    def root_log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the root log-likelihood ratio for two events.
        """
        llr = LogLikelihood.log_likelihood_ratio(k11, k12, k21, k22)
        sqrt = math.sqrt(llr)
        if (k11 / (k11 + k12)) < (k21 / (k21 + k22)):
            sqrt = -sqrt
        return sqrt

    @staticmethod
    def compare_frequencies(a: Counter, b: Counter, max_return: int, threshold: float) -> List[Tuple[Any, float]]:
        """
        Compares two sets of counts to find over-represented items in the first set.
        """
        total_a = sum(a.values())
        total_b = sum(b.values())
        best = []

        def compare_and_add(t):
            k_a = a[t]
            k_b = b[t]
            score = LogLikelihood.root_log_likelihood_ratio(k_a, total_a - k_a, k_b, total_b - k_b)
            if score >= threshold:
                heappush(best, (score, t))
                if len(best) > max_return:
                    heappop(best)

        for t in a.keys():
            compare_and_add(t)

        if threshold < 0:
            for t in b.keys():
                if t not in a:
                    compare_and_add(t)

        # Sort results by score descending
        return sorted(best, key=lambda x: -x[0])

class ScoredItem:
    def __init__(self, item, score):
        self.item = item
        self.score = score

    def __repr__(self):
        return f"ScoredItem(item={self.item}, score={self.score})"


Y realizamos la llamada al método compare_frequencies pasándole:

1.   La lista de frecuencias de los términos de los documentos relevantes

2.   La lista de frecuencias de los términos del resto de documentos

3.   Un valor de +infinito para indicar al método que queremos obtener TODOS los valores LLR con signo que se calculen.

4.   Un valor de -infinito para indicar al método que no queremos un valor mínimo para que se incluya una puntuación o no en la lista de valores LLR con signo resultantes.

In [ ]:
numeroMaximoDeTerminos = float('inf') # para que se incluyan todas las puntuaciones
limiteDePuntuacion = -float('inf') # para que no haya un valor minimo a la hora de puntuar

conjuntoValoresLLRSigno = []

for i in range(len(frecuencias_relevantes)):
  valoresLLRSigno = LogLikelihood.compare_frequencies(Counter(frecuencias_relevantes[i]), Counter(frecuencias_relevantes_restantes[i]), numeroMaximoDeTerminos, limiteDePuntuacion)
  conjuntoValoresLLRSigno.append(valoresLLRSigno)
  #print(valoresLLRSigno)

Seleccionamos los m términos con los valores más altos que, además, no aparezcan en la consulta original.

In [ ]:
m = 3 # parámetro configurable
mTerminosRelevantesConLosValoresMasAltos = []

for i in range(len(queries)):
  mTerminosRelevantesConLosValoresMasAltos.append(dict())

frecuenciasTerminosQueries = dict()

index = 0
for query in queries:
  frecuenciasTerminosQueries[query["_id"]] = calcularTF(queries[index]["metadata"]["narrative"], True)

  numeroDeTerminosRelevantesEncontrados = 0
  indexValoresLLRS = 0
  while indexValoresLLRS < len(conjuntoValoresLLRSigno[index]) and numeroDeTerminosRelevantesEncontrados < m:
    tupla = conjuntoValoresLLRSigno[index][indexValoresLLRS]
    valor = tupla[0]
    termino = tupla[1]

    if termino not in frecuenciasTerminosQueries[query["_id"]].keys():
      mTerminosRelevantesConLosValoresMasAltos[index][termino] = valor
      numeroDeTerminosRelevantesEncontrados += 1

    indexValoresLLRS += 1

  index += 1

for i in range(len(mTerminosRelevantesConLosValoresMasAltos)):
  print("m =", m, "terminos que no salen en la consulta inicial", i+1,":")
  for j in mTerminosRelevantesConLosValoresMasAltos[i]:
    print("Término:", j, "\tValor:", mTerminosRelevantesConLosValoresMasAltos[i][j])
  print()

Los m términos obtenidos los añadimos a la consulta inicial

In [ ]:
queriesExpandidas = queries

index = 0
for consulta in queriesExpandidas:
    terminosAñadir = ""
    for termino in mTerminosRelevantesConLosValoresMasAltos[index]:
      terminosAñadir += " " + termino
    index += 1
    print("Antes:", consulta["metadata"]["narrative"])
    consulta["metadata"]["narrative"] += terminosAñadir
    print("Despues:", consulta["metadata"]["narrative"])
    print()

Y volvemos a enviar al índice BM25S la consulta para obtener la lista definitiva de resultados.

In [ ]:
resultadosFinales = submit_queries_and_get_run(queriesExpandidas, stemmer, retriever, "en", 100, True)
for i in resultadosFinales:
  print()
  print("Query", i, ":")
  for resultado in resultadosFinales[i]:
    print("Documento:", resultado[0]["id"], "\tPuntuación:", resultado[1])